## Monte Carlo simulation to see how much the SN distributions can vary
We want to see how small sample sizes affect our distributions. We do Monte Carlo simulations to see this variation.

In [ ]:
import numpy as np
import pandas as pd
import math
from astropy.utils.console import ProgressBar
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from scipy.stats import ks_2samp
import seaborn as sns
from scipy.stats import anderson_ksamp

In [ ]:
sn_conv2p0 = pd.read_csv('../sne_project/sample_processing/2kpc.csv')
pgcs  = sn_conv2p0['PGC']

In [ ]:
bands = ['W1','W2','W3','W4','NUV','FUV','SFR_FUVW4','SFR_NUVW3']
sne   = ['Ia','II','Ibc']

result_sfr = pd.read_csv('../sne_project/results/ks_test/SFR_2kpc.csv')
result     = pd.read_csv('../sne_project/results/ks_test/2kpc.csv')


# for each band
for band in range(len(bands)):
    print(bands[band])
    
    # for each SN type in each band
    for tipe in range(len(sne)):
        print(sne[tipe])
        distributions = pd.DataFrame()
        # build 100 plots per SN type in each band
        bar = ProgressBar(range(100), ipython_widget=True)
        for j in range(0, 100):
            pix = []
            
            if (bands[band] == 'SFR_FUVW4') or (bands[band] == 'SFR_NUVW3'):
                result = result_sfr

            num_sne = result['%s_%s' % (bands[band],sne[tipe])].dropna()
            # for each individual plot per SN type per band
            for i in range(0, len(num_sne)):

               # loop until we find a galaxy that has a control table
                random_gal = np.random.choice(pgcs)
                while os.path.isfile('../sne_project/galaxy_control_tables/2kpc/%s_2kpc.csv' % (random_gal)) is not True:
                    random_gal = np.random.choice(pgcs)

                control_table = pd.read_csv('../sne_project/galaxy_control_tables/2kpc/%s_2kpc.csv' % (random_gal))
                sne_flux = np.array(control_table['%s'% (bands[band])])
                is_positive = np.where(sne_flux >= 0)

                # loop until we find a galaxy that has a control table *and* an FUV column
                while np.size(is_positive) == 0:
                    random_gal = np.random.choice(pgcs)
                    while os.path.isfile('../sne_project/galaxy_control_tables/2kpc/%s_2kpc.csv'% (random_gal)) is not True:
                        random_gal = np.random.choice(pgcs)

                    control_table = pd.read_csv('../sne_project/galaxy_control_tables/2kpc/%s_2kpc.csv' % (random_gal))
                    sne_flux = np.array(control_table['%s'% (bands[band])])
                    is_positive = np.where(sne_flux >= 0)

                band_flux = np.array(control_table['%s' % (bands[band])])
                
                if (bands[band] == 'W4') or (bands[band] == 'SFR_FUVW4'):
                    band_rad  = np.array(control_table['RADIUS_W4'])
                else:
                    band_rad  = np.array(control_table['RADIUS'])

    
                # get random pixel's radius                          # normalized pixel value
                random_rad = np.random.choice(a=band_rad[is_positive], p=sne_flux[is_positive]/np.sum(sne_flux[is_positive]))
                gal_cdf = pd.read_csv('../sne_project/galaxy_control_tables/gal_cdfs_2kpc/%s_%s_2kpc.csv' % (random_gal, bands[band]))
                radii   = np.array(gal_cdf['RADIUS'])
                cdf_val = np.array(gal_cdf['CDF'])
                random_cdf = np.interp(random_rad, radii, cdf_val)
                pix.append(random_cdf)
            bar.update()

            pix = np.array(pix)
            pix = pix[np.isfinite(pix)]
            pix = np.sort(pix)
            distributions['DIST_%s' % (j)] = pix

        distributions.to_csv('../sne_project/results/monte_carlo/%s_%s_2kpc.csv' % (bands[band],sne[tipe]),index=False)